### Import required libraries

In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import cv2
import matplotlib.pyplot as plt

### Load data

In [2]:
print('Loading data')
x = pd.read_csv("../data/train_x.csv", delimiter=",", header=None)
print(' Loaded x')

Loading data
 Loaded x


In [3]:
x = x.values
x = x.reshape(-1, 64, 64)
print(x.shape)

NameError: name 'y' is not defined

In [ ]:
# Threshold for the digit only

binary_x = ((x > 254) + 0) * 255
binary_x = binary_x.astype(np.uint8)
print(binary_x.shape, binary_x.dtype)

In [ ]:
for i in range(10):
    plt.imshow(binary_x[i])
    plt.show()

### Segment largest digit

In [ ]:
def biggest_bounding_box(img, isSquare=False):
    
    ret, thresh = cv2.threshold(img, 1, 255,0)
    img, contours, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    xs, ys, ws, hs = [], [], [], []
    areas = []
    
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        # print(x, y, w, h)
        
        xs.append(x); ys.append(y); ws.append(w); hs.append(h)
        
        if isSquare:
            length = max(w,h)
            areas.append(length * length)
        else:
            areas.append(w * h)
    
    areas = np.array(areas)
    biggest = np.argmax(areas)

    x = xs[biggest]
    y = ys[biggest]
    w = ws[biggest]
    h = hs[biggest]
    
    # print(x, y, w, h)
    # plt.imshow(cv2.rectangle(img, (x,y), (x+w,y+h), (100,100,100), 1))
    # plt.show()
    
    return img[y:y+h, x:x+w]

In [ ]:
extracted_x = []

for i, p in enumerate(binary_x):
    
    if i % 2000 == 0:
        print(i)
    
    extracted_x.append(biggest_bounding_box(p))

In [ ]:
centered_x = np.zeros(np.shape(binary_x))

for i, d in enumerate(extracted_x):
    
    x, y = np.shape(d)
    x_pos = int(32 - (x/2))
    y_pos = int(32 - (y/2))
    
    centered_x[i][x_pos: x_pos + x, y_pos: y_pos + y] = d

In [ ]:
for i in range(10):
    plt.imshow(centered_x[i])
    plt.show()

In [ ]:
np.save("../data/x_centred_train.npy", centered_x)